<a href="https://colab.research.google.com/github/ShaunakSoni28/Age-Gender-Emotion-Detection/blob/master/Bubble_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install networkx plotly numpy dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 144.1 MB/s eta 0:00:00


In [2]:
import plotly.graph_objects as go
import networkx as nx
import numpy as np
from plotly.subplots import make_subplots

class InteractiveBubbleNetwork:
    def __init__(self):
        self.graph = nx.Graph()
        self.node_sizes = {}
        self.node_colors = {}
        self.node_labels = {}
        self.node_info = {}

    def add_node(self, node_id, size=20, color='lightblue', label=None, info=None):
        """Add a node to the network with customizable properties"""
        self.graph.add_node(node_id)
        self.node_sizes[node_id] = size
        self.node_colors[node_id] = color
        self.node_labels[node_id] = label if label else str(node_id)
        self.node_info[node_id] = info if info else f"Node {node_id}"

    def add_edge(self, node1, node2, weight=1):
        """Add an edge between two nodes"""
        self.graph.add_edge(node1, node2, weight=weight)

    def create_visualization(self, layout='spring', title="Interactive Bubble Network"):
        """Create an interactive plotly visualization"""
        # Get layout positions
        if layout == 'spring':
            pos = nx.spring_layout(self.graph, k=2, iterations=50)
        elif layout == 'circular':
            pos = nx.circular_layout(self.graph)
        elif layout == 'kamada_kawai':
            pos = nx.kamada_kawai_layout(self.graph)
        else:
            pos = nx.random_layout(self.graph)

        # Create edge traces
        edge_trace = []
        for edge in self.graph.edges():
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edge_trace.append(go.Scatter(
                x=[x0, x1, None],
                y=[y0, y1, None],
                mode='lines',
                line=dict(width=0.5, color='#888'),
                hoverinfo='none',
                showlegend=False
            ))

        # Create node trace
        node_x = []
        node_y = []
        node_sizes_list = []
        node_colors_list = []
        node_text = []
        hover_text = []

        for node in self.graph.nodes():
            x, y = pos[node]
            node_x.append(x)
            node_y.append(y)
            node_sizes_list.append(self.node_sizes.get(node, 20))
            node_colors_list.append(self.node_colors.get(node, 'lightblue'))
            node_text.append(self.node_labels.get(node, str(node)))

            # Create hover text with node info and connections
            connections = list(self.graph.neighbors(node))
            hover = f"<b>{self.node_labels.get(node, str(node))}</b><br>"
            hover += f"{self.node_info.get(node, '')}<br>"
            hover += f"Connections: {len(connections)}<br>"
            if connections:
                hover += f"Connected to: {', '.join([self.node_labels.get(n, str(n)) for n in connections[:5]])}"
                if len(connections) > 5:
                    hover += f"... and {len(connections)-5} more"
            hover_text.append(hover)

        node_trace = go.Scatter(
            x=node_x,
            y=node_y,
            mode='markers+text',
            marker=dict(
                size=node_sizes_list,
                color=node_colors_list,
                line=dict(width=2, color='white'),
                opacity=0.9
            ),
            text=node_text,
            textposition="top center",
            hovertext=hover_text,
            hoverinfo='text',
            hoverlabel=dict(bgcolor="white"),
            showlegend=False
        )

        # Create figure
        fig = go.Figure(data=edge_trace + [node_trace])

        # Update layout
        fig.update_layout(
            title=title,
            titlefont_size=16,
            showlegend=False,
            hovermode='closest',
            margin=dict(b=20, l=5, r=5, t=40),
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            plot_bgcolor='white',
            height=600,
            clickmode='event+select'
        )

        return fig

    def show(self, **kwargs):
        """Display the network"""
        fig = self.create_visualization(**kwargs)
        fig.show()

    def get_network_stats(self):
        """Get basic network statistics"""
        stats = {
            "Number of nodes": self.graph.number_of_nodes(),
            "Number of edges": self.graph.number_of_edges(),
            "Average degree": sum(dict(self.graph.degree()).values()) / self.graph.number_of_nodes() if self.graph.number_of_nodes() > 0 else 0,
            "Density": nx.density(self.graph),
            "Is connected": nx.is_connected(self.graph)
        }
        if self.graph.number_of_nodes() > 0 and nx.is_connected(self.graph):
            stats["Diameter"] = nx.diameter(self.graph)
            stats["Average clustering"] = nx.average_clustering(self.graph)
        return stats


# Example 1: Social Network
def create_social_network():
    network = InteractiveBubbleNetwork()

    # Add people as nodes with different sizes based on influence
    people = {
        "Alice": {"size": 40, "color": "lightcoral", "info": "Team Lead, 5 years experience"},
        "Bob": {"size": 30, "color": "lightblue", "info": "Developer, 3 years experience"},
        "Charlie": {"size": 35, "color": "lightgreen", "info": "Designer, 4 years experience"},
        "Diana": {"size": 25, "color": "plum", "info": "Junior Developer, 1 year experience"},
        "Eve": {"size": 30, "color": "lightsalmon", "info": "Product Manager, 3 years experience"},
        "Frank": {"size": 20, "color": "lightgray", "info": "Intern, 6 months"},
    }

    for person, props in people.items():
        network.add_node(person, **props, label=person)

    # Add relationships
    relationships = [
        ("Alice", "Bob"), ("Alice", "Charlie"), ("Alice", "Diana"),
        ("Bob", "Charlie"), ("Bob", "Diana"),
        ("Charlie", "Eve"), ("Diana", "Frank"),
        ("Eve", "Alice"), ("Eve", "Frank")
    ]

    for rel in relationships:
        network.add_edge(rel[0], rel[1])

    return network


# Example 2: Topic/Concept Network
def create_topic_network():
    network = InteractiveBubbleNetwork()

    # Add topics with sizes based on importance
    topics = {
        "Machine Learning": {"size": 50, "color": "gold", "info": "Core AI technology"},
        "Deep Learning": {"size": 40, "color": "orange", "info": "Subset of ML using neural networks"},
        "NLP": {"size": 35, "color": "lightcoral", "info": "Natural Language Processing"},
        "Computer Vision": {"size": 35, "color": "lightblue", "info": "Image and video analysis"},
        "Reinforcement Learning": {"size": 30, "color": "lightgreen", "info": "Learning through rewards"},
        "Transformers": {"size": 30, "color": "plum", "info": "Architecture for sequence modeling"},
        "CNNs": {"size": 25, "color": "lightsalmon", "info": "Convolutional Neural Networks"},
        "RNNs": {"size": 25, "color": "lightcyan", "info": "Recurrent Neural Networks"},
        "GANs": {"size": 25, "color": "lightpink", "info": "Generative Adversarial Networks"},
    }

    for topic, props in topics.items():
        network.add_node(topic, **props, label=topic)

    # Add connections between related topics
    connections = [
        ("Machine Learning", "Deep Learning"),
        ("Deep Learning", "NLP"),
        ("Deep Learning", "Computer Vision"),
        ("Deep Learning", "Reinforcement Learning"),
        ("NLP", "Transformers"),
        ("NLP", "RNNs"),
        ("Computer Vision", "CNNs"),
        ("Deep Learning", "GANs"),
        ("Transformers", "RNNs"),
    ]

    for conn in connections:
        network.add_edge(conn[0], conn[1])

    return network


# Example 3: Dynamic network builder
class DynamicNetworkBuilder:
    def __init__(self):
        self.network = InteractiveBubbleNetwork()

    def add_cluster(self, center_node, connected_nodes, center_color="red", node_color="lightblue"):
        """Add a cluster of nodes around a center node"""
        # Add center node
        self.network.add_node(
            center_node,
            size=40,
            color=center_color,
            label=center_node,
            info=f"Central hub: {center_node}"
        )

        # Add connected nodes
        for node in connected_nodes:
            self.network.add_node(
                node,
                size=20,
                color=node_color,
                label=node,
                info=f"Connected to {center_node}"
            )
            self.network.add_edge(center_node, node)

    def connect_clusters(self, node1, node2):
        """Connect two nodes from different clusters"""
        self.network.add_edge(node1, node2)

    def show(self, **kwargs):
        return self.network.show(**kwargs)


# Usage examples
if __name__ == "__main__":
    # Example 1: Create and display social network
    print("Creating Social Network...")
    social = create_social_network()
    print("Network Stats:", social.get_network_stats())
    social.show(layout='spring', title="Team Social Network")

    # Example 2: Create and display topic network
    print("\nCreating Topic Network...")
    topics = create_topic_network()
    print("Network Stats:", topics.get_network_stats())
    topics.show(layout='kamada_kawai', title="Machine Learning Topics Network")

    # Example 3: Build a dynamic network
    print("\nCreating Dynamic Network...")
    builder = DynamicNetworkBuilder()

    # Add multiple clusters
    builder.add_cluster("Hub A", ["A1", "A2", "A3"], "red", "lightcoral")
    builder.add_cluster("Hub B", ["B1", "B2", "B3"], "blue", "lightblue")
    builder.add_cluster("Hub C", ["C1", "C2"], "green", "lightgreen")

    # Connect clusters
    builder.connect_clusters("A2", "B1")
    builder.connect_clusters("B3", "C1")
    builder.connect_clusters("Hub A", "Hub B")

    builder.show(layout='spring', title="Dynamic Multi-Cluster Network")

Creating Social Network...
Network Stats: {'Number of nodes': 6, 'Number of edges': 9, 'Average degree': 3.0, 'Density': 0.6, 'Is connected': True, 'Diameter': 2, 'Average clustering': 0.4166666666666667}



Creating Topic Network...
Network Stats: {'Number of nodes': 9, 'Number of edges': 9, 'Average degree': 2.0, 'Density': 0.25, 'Is connected': True, 'Diameter': 4, 'Average clustering': 0.2592592592592593}



Creating Dynamic Network...


In [3]:
# Create a network
network = InteractiveBubbleNetwork()

# Add nodes with properties
network.add_node("Node1", size=30, color="lightblue", info="Description")

# Add edges
network.add_edge("Node1", "Node2")

# Display
network.show(layout='spring', title="My Network")

In [15]:
import plotly.graph_objects as go
import networkx as nx
import numpy as np
import json
from IPython.display import display, HTML
import ipywidgets as widgets
from plotly.offline import init_notebook_mode, iplot
import plotly.io as pio

# Initialize plotly for notebook
init_notebook_mode(connected=True)

class InteractiveDraggableNetwork:
    def __init__(self):
        self.graph = nx.Graph()
        self.node_sizes = {}
        self.node_colors = {}
        self.node_labels = {}
        self.node_info = {}
        self.positions = {}
        self.fixed_positions = {}

    def add_node(self, node_id, size=20, color='lightblue', label=None, info=None):
        """Add a node to the network with customizable properties"""
        self.graph.add_node(node_id)
        self.node_sizes[node_id] = size
        self.node_colors[node_id] = color
        self.node_labels[node_id] = label if label else str(node_id)
        self.node_info[node_id] = info if info else f"Node {node_id}"

    def add_edge(self, node1, node2, weight=1):
        """Add an edge between two nodes"""
        self.graph.add_edge(node1, node2, weight=weight)

    def get_layout_positions(self, layout='spring'):
        """Calculate initial positions"""
        if layout == 'spring':
            self.positions = nx.spring_layout(self.graph, k=2, iterations=50, scale=2)
        elif layout == 'circular':
            self.positions = nx.circular_layout(self.graph, scale=2)
        elif layout == 'kamada_kawai':
            self.positions = nx.kamada_kawai_layout(self.graph, scale=2)
        else:
            self.positions = nx.random_layout(self.graph, scale=2)
        return self.positions

    def create_plotly_figure(self, layout='spring', title="Interactive Network"):
        """Create an interactive plotly figure with proper drag support"""
        pos = self.get_layout_positions(layout)

        # Store positions for updates
        self.current_positions = pos.copy()

        # Create figure
        fig = go.Figure()

        # Add edges first (so they appear behind nodes)
        for edge in self.graph.edges():
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]

            fig.add_trace(go.Scatter(
                x=[x0, x1, None],
                y=[y0, y1, None],
                mode='lines',
                line=dict(width=1, color='rgba(125,125,125,0.3)'),
                hoverinfo='skip',
                showlegend=False,
                name=f'edge_{edge[0]}_{edge[1]}'
            ))

        # Add nodes as individual traces for better control
        node_x = []
        node_y = []
        node_text = []
        node_sizes = []
        node_colors = []
        hover_texts = []

        for node in self.graph.nodes():
            x, y = pos[node]
            node_x.append(x)
            node_y.append(y)
            node_text.append(self.node_labels.get(node, str(node)))
            node_sizes.append(self.node_sizes.get(node, 20))
            node_colors.append(self.node_colors.get(node, 'lightblue'))

            # Create hover text
            connections = list(self.graph.neighbors(node))
            hover = f"<b>{self.node_labels.get(node, str(node))}</b><br>"
            hover += f"{self.node_info.get(node, '')}<br>"
            hover += f"<b>Connections:</b> {len(connections)}<br>"
            if connections:
                connected_names = [self.node_labels.get(n, str(n)) for n in connections[:5]]
                hover += f"<b>Connected to:</b> {', '.join(connected_names)}"
                if len(connections) > 5:
                    hover += f"... and {len(connections)-5} more"
            hover_texts.append(hover)

        # Add all nodes as one trace
        fig.add_trace(go.Scatter(
            x=node_x,
            y=node_y,
            mode='markers+text',
            marker=dict(
                size=node_sizes,
                color=node_colors,
                line=dict(width=2, color='white'),
                opacity=0.9
            ),
            text=node_text,
            textposition="top center",
            textfont=dict(size=10),
            hovertext=hover_texts,
            hoverinfo='text',
            hoverlabel=dict(bgcolor="white"),
            showlegend=False,
            name='nodes'
        ))

        # Update layout for better interaction
        fig.update_layout(
            title=title,
            showlegend=False,
            hovermode='closest',
            margin=dict(b=20, l=20, r=20, t=40),
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            plot_bgcolor='#f8f9fa',
            height=600,
            dragmode='pan',
            clickmode='event+select',
        )

        return fig

    def create_cytoscape_network(self, layout='spring', title="Draggable Network"):
        """Create a Cytoscape-style network using ipycytoscape for true node dragging"""
        try:
            import ipycytoscape
        except ImportError:
            print("Installing ipycytoscape...")
            import subprocess
            import sys
            subprocess.check_call([sys.executable, "-m", "pip", "install", "ipycytoscape"])
            import ipycytoscape

        # Get positions
        pos = self.get_layout_positions(layout)

        # Create cytoscape widget
        cyto = ipycytoscape.CytoscapeWidget()
        cyto.graph.add_nodes([
            {
                'data': {
                    'id': str(node),
                    'label': self.node_labels.get(node, str(node)),
                    'size': self.node_sizes.get(node, 20),
                    'color': self.node_colors.get(node, 'lightblue')
                },
                'position': {
                    'x': pos[node][0] * 300 + 400,
                    'y': pos[node][1] * 300 + 300
                }
            }
            for node in self.graph.nodes()
        ])

        # Add edges
        cyto.graph.add_edges([
            {'data': {'source': str(edge[0]), 'target': str(edge[1])}}
            for edge in self.graph.edges()
        ])

        # Set style
        cyto.set_style([
            {
                'selector': 'node',
                'style': {
                    'background-color': 'data(color)',
                    'label': 'data(label)',
                    'width': 'data(size)',
                    'height': 'data(size)',
                    'text-valign': 'top',
                    'text-halign': 'center',
                    'font-size': '12px'
                }
            },
            {
                'selector': 'edge',
                'style': {
                    'line-color': '#888',
                    'width': 1,
                    'curve-style': 'straight'
                }
            },
            {
                'selector': ':selected',
                'style': {
                    'background-color': '#ff0000',
                    'line-color': '#ff0000'
                }
            }
        ])

        # Set layout
        cyto.set_layout(name='preset')

        return cyto

    def create_vis_network(self, layout='spring', title="Draggable Network", height='600px'):
        """Create a vis.js network with true drag-and-drop using pyvis"""
        try:
            from pyvis.network import Network
        except ImportError:
            print("Installing pyvis...")
            import subprocess
            import sys
            subprocess.check_call([sys.executable, "-m", "pip", "install", "pyvis"])
            from pyvis.network import Network

        # Create pyvis network
        net = Network(height=height, width='100%', bgcolor='#f8f9fa', font_color='black')

        # Get positions
        pos = self.get_layout_positions(layout)

        # Add nodes with positions
        for node in self.graph.nodes():
            x_pos = pos[node][0] * 500
            y_pos = pos[node][1] * 500

            net.add_node(
                str(node),
                label=self.node_labels.get(node, str(node)),
                title=self.node_info.get(node, ''),
                size=self.node_sizes.get(node, 20),
                color=self.node_colors.get(node, 'lightblue'),
                x=x_pos,
                y=y_pos,
                physics=False  # Disable physics for manual positioning
            )

        # Add edges
        for edge in self.graph.edges():
            net.add_edge(str(edge[0]), str(edge[1]), color='#888', width=1)

        # Set options for better interaction
        net.set_options("""
        var options = {
            "nodes": {
                "borderWidth": 2,
                "borderWidthSelected": 3,
                "shape": "dot",
                "font": {
                    "size": 12
                }
            },
            "edges": {
                "smooth": {
                    "type": "continuous"
                }
            },
            "physics": {
                "enabled": false
            },
            "interaction": {
                "dragNodes": true,
                "dragView": true,
                "zoomView": true,
                "hover": true
            }
        }
        """)

        return net

    def show_interactive(self, method='vis', layout='spring', title="Interactive Network"):
        """Show the network with different visualization methods"""

        if method == 'vis':
            # Use pyvis for best drag-and-drop
            net = self.create_vis_network(layout, title)

            # Save and display
            net.save_graph('network.html')

            # For Colab
            try:
                from google.colab import files
                display(HTML("""
                <div style="background: white; padding: 10px; border-radius: 5px;">
                    <h3>Network saved as 'network.html'</h3>
                    <p>Click the button below to download and open locally for full interaction:</p>
                </div>
                """))
                files.download('network.html')

                # Also display inline
                display(HTML(net.html))
            except:
                # For Jupyter/local
                display(HTML(net.html))
                print(f"Network saved as 'network.html'. Open this file in a browser for full interaction.")

        elif method == 'plotly':
            # Use enhanced plotly with better controls
            fig = self.create_plotly_figure(layout, title)

            # Add custom JS for better dragging (experimental)
            fig.update_layout(
                updatemenus=[
                    dict(
                        buttons=list([
                            dict(
                                args=["dragmode", "pan"],
                                label="Pan",
                                method="relayout"
                            ),
                            dict(
                                args=["dragmode", "select"],
                                label="Select",
                                method="relayout"
                            ),
                            dict(
                                args=["dragmode", "lasso"],
                                label="Lasso",
                                method="relayout"
                            ),
                        ]),
                        direction="down",
                        showactive=True,
                        x=0.1,
                        y=1.15
                    ),
                ]
            )

            # Show with config for editing
            config = {
                'editable': True,
                'edits': {
                    'shapePosition': True,
                    'annotationPosition': True,
                    'annotationTail': True,
                    'annotationText': True
                },
                'toImageButtonOptions': {
                    'format': 'png',
                    'filename': 'network_graph',
                    'height': 800,
                    'width': 1200,
                    'scale': 1
                }
            }

            fig.show(config=config)

        elif method == 'cytoscape':
            # Use ipycytoscape for Jupyter environments
            cyto = self.create_cytoscape_network(layout, title)
            display(HTML(f"<h3>{title}</h3>"))
            display(cyto)

        else:
            print("Method not recognized. Use 'vis', 'plotly', or 'cytoscape'")

    def show_in_colab(self, layout='spring', title="Interactive Network"):
        """Optimized for Google Colab - creates downloadable HTML with full drag-drop"""
        net = self.create_vis_network(layout, title, height='700px')

        # Enhanced HTML with instructions
        html_content = net.generate_html()

        # Save file
        with open('interactive_network.html', 'w') as f:
            f.write(html_content)

        # Display in Colab
        try:
            from google.colab import files
            from IPython.display import IFrame

            # Display instructions
            display(HTML("""
            <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                        color: white; padding: 20px; border-radius: 10px; margin: 10px 0;">
                <h2 style="margin: 0 0 10px 0;">🎨 Interactive Network Ready!</h2>
                <p style="margin: 5px 0;"><b>Option 1:</b> Click the download button below to save and open locally for best experience</p>
                <p style="margin: 5px 0;"><b>Option 2:</b> View inline below (may have limited interaction in Colab)</p>
                <p style="margin: 10px 0 0 0;">✨ <b>Features:</b> Drag nodes • Zoom • Pan • Hover for details</p>
            </div>
            """))

            # Provide download
            files.download('interactive_network.html')

            # Try to display inline
            display(IFrame(src='interactive_network.html', width='100%', height='600'))

        except ImportError:
            # For non-Colab environments
            display(HTML(html_content))
            print("Network saved as 'interactive_network.html'. Open in browser for full interaction.")


# Example functions
def create_social_network():
    """Create an example social network"""
    network = InteractiveDraggableNetwork()

    # Add team members
    team = {
        "Alice": {"size": 50, "color": "#FF6B6B", "info": "Team Lead - 5 years experience"},
        "Bob": {"size": 40, "color": "#4ECDC4", "info": "Senior Developer - 3 years"},
        "Charlie": {"size": 35, "color": "#45B7D1", "info": "Designer - 4 years"},
        "Diana": {"size": 30, "color": "#96CEB4", "info": "Junior Developer - 1 year"},
        "Eve": {"size": 35, "color": "#FFEAA7", "info": "Product Manager - 3 years"},
        "Frank": {"size": 25, "color": "#DDA0DD", "info": "Intern - 6 months"},
        "Grace": {"size": 35, "color": "#98D8C8", "info": "QA Lead - 4 years"},
        "Henry": {"size": 30, "color": "#FFD93D", "info": "DevOps Engineer - 2 years"}
    }

    for person, props in team.items():
        network.add_node(person, **props, label=person)

    # Add relationships
    edges = [
        ("Alice", "Bob"), ("Alice", "Charlie"), ("Alice", "Diana"), ("Alice", "Eve"),
        ("Bob", "Charlie"), ("Bob", "Diana"), ("Bob", "Henry"),
        ("Charlie", "Eve"), ("Charlie", "Grace"),
        ("Diana", "Frank"), ("Diana", "Henry"),
        ("Eve", "Grace"), ("Eve", "Frank"),
        ("Grace", "Henry"), ("Frank", "Henry")
    ]

    for edge in edges:
        network.add_edge(edge[0], edge[1])

    return network


def create_topic_network():
    """Create a topic/concept network"""
    network = InteractiveDraggableNetwork()

    # Add topics
    topics = {
        "Machine Learning": {"size": 60, "color": "#FFD700", "info": "Core AI technology"},
        "Deep Learning": {"size": 50, "color": "#FF8C00", "info": "Neural networks"},
        "NLP": {"size": 40, "color": "#FF69B4", "info": "Natural Language Processing"},
        "Computer Vision": {"size": 40, "color": "#00CED1", "info": "Image analysis"},
        "Reinforcement Learning": {"size": 35, "color": "#32CD32", "info": "Learning through rewards"},
        "Transformers": {"size": 35, "color": "#9370DB", "info": "Attention mechanism"},
        "CNNs": {"size": 30, "color": "#FF6347", "info": "Convolutional Neural Networks"},
        "RNNs": {"size": 30, "color": "#4682B4", "info": "Recurrent Neural Networks"},
        "GANs": {"size": 30, "color": "#DA70D6", "info": "Generative Adversarial Networks"}
    }

    for topic, props in topics.items():
        network.add_node(topic, **props, label=topic)

    # Add connections
    connections = [
        ("Machine Learning", "Deep Learning"),
        ("Deep Learning", "NLP"),
        ("Deep Learning", "Computer Vision"),
        ("Deep Learning", "Reinforcement Learning"),
        ("NLP", "Transformers"),
        ("NLP", "RNNs"),
        ("Computer Vision", "CNNs"),
        ("Deep Learning", "GANs"),
        ("Transformers", "RNNs"),
        ("CNNs", "GANs")
    ]

    for conn in connections:
        network.add_edge(conn[0], conn[1])

    return network


# Main execution
if __name__ == "__main__":
    print("🎨 Interactive Draggable Network Generator")
    print("=" * 50)

    # Check if in Colab
    try:
        from google.colab import files
        in_colab = True
        print("📍 Running in Google Colab")
    except:
        in_colab = False
        print("📍 Running in local environment")

    print("\nSelect network type:")
    print("1. Social Network (Team collaboration)")
    print("2. Topic Network (Machine Learning concepts)")
    print("3. Custom Network")

    choice = input("\nEnter choice (1/2/3): ").strip()

    if choice == "1":
        network = create_social_network()
        title = "Team Collaboration Network"
    elif choice == "2":
        network = create_topic_network()
        title = "Machine Learning Concepts Network"
    else:
        # Create custom network
        network = InteractiveDraggableNetwork()
        n_nodes = int(input("Number of nodes (e.g., 5): ") or 5)

        for i in range(1, n_nodes + 1):
            network.add_node(
                f"Node{i}",
                size=30 + i*5,
                color=f"hsl({i*360/n_nodes}, 70%, 60%)",
                label=f"N{i}",
                info=f"Custom node {i}"
            )

        # Add some random edges
        import random
        for _ in range(n_nodes):
            n1 = f"Node{random.randint(1, n_nodes)}"
            n2 = f"Node{random.randint(1, n_nodes)}"
            if n1 != n2:
                network.add_edge(n1, n2)

        title = "Custom Network"

    print("\nSelect visualization method:")
    print("1. Vis.js (Best drag-and-drop, downloads HTML)")
    print("2. Plotly (Good interactivity, inline display)")
    print("3. Auto (Best for your environment)")

    method_choice = input("\nEnter choice (1/2/3): ").strip()

    if method_choice == "1":
        method = 'vis'
    elif method_choice == "2":
        method = 'plotly'
    else:
        method = 'vis' if in_colab else 'plotly'

    print(f"\n🚀 Creating {title}...")

    if in_colab:
        network.show_in_colab(layout='spring', title=title)
    else:
        network.show_interactive(method=method, layout='spring', title=title)

    print("\n✅ Network created successfully!")
    if method == 'vis':
        print("📁 Check 'interactive_network.html' for the full interactive experience")

🎨 Interactive Draggable Network Generator
📍 Running in Google Colab

Select network type:
1. Social Network (Team collaboration)
2. Topic Network (Machine Learning concepts)
3. Custom Network

Enter choice (1/2/3): 2

Select visualization method:
1. Vis.js (Best drag-and-drop, downloads HTML)
2. Plotly (Good interactivity, inline display)
3. Auto (Best for your environment)

Enter choice (1/2/3): 2

🚀 Creating Machine Learning Concepts Network...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Network created successfully!
